<a href="https://colab.research.google.com/github/rafaelhamer/DSWP/blob/master/Labdata_Churn_parte_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

def mostra_missing_value(df):
    total = df.isnull().sum().sort_values(ascending = False)
    percent = 100*round((df.isnull().sum()/df.isnull().count()).sort_values(ascending = False), 2)
    missing_data = pd.concat([total, percent], axis = 1, keys=['Total', 'Percentual'])
    print(missing_data.head(10))

from google.colab import drive
drive.mount('/gdrive')

## Lendo os dados de Treino e Teste

In [81]:
df_train = pd.read_csv('/gdrive/My Drive/Python/treino_limpo2.csv', index_col='id')
df_test  = pd.read_csv('/gdrive/My Drive/Python/teste_limpo2.csv', index_col=0)

submission = pd.read_csv('/gdrive/My Drive/Python/sample_submission.csv')

In [59]:
df_train.dtypes

SeniorCitizen             int64
tenure                  float64
MonthlyCharges          float64
Churn                     int64
Partner_num               int64
Dependents_num            int64
PhoneService_num          int64
MultipleLines_num         int64
InternetService_num       int64
OnlineSecurity_num        int64
OnlineBackup_num          int64
DeviceProtection_num      int64
TechSupport_num           int64
StreamingTV_num           int64
StreamingMovies_num       int64
Contract_num              int64
PaperlessBilling_num      int64
PaymentMethod_num         int64
dtype: object

In [ ]:
mostra_missing_value(df_train)

In [ ]:
mostra_missing_value(df_test)

In [60]:
df_train.head()

,SeniorCitizen,tenure,MonthlyCharges,Churn,Partner_num,Dependents_num,PhoneService_num,MultipleLines_num,InternetService_num,OnlineSecurity_num,OnlineBackup_num,DeviceProtection_num,TechSupport_num,StreamingTV_num,StreamingMovies_num,Contract_num,PaperlessBilling_num,PaymentMethod_num
id,,,,,,,,,,,,,,,,,,
4030,0,56.000000,45.05,0,0,0,0,1,0,2,2,2,2,0,0,2,0,4
6731,0,16.928753,19.65,0,1,1,1,0,2,1,1,1,1,1,1,1,0,3
6479,0,60.000000,110.80,0,1,0,1,2,1,0,2,2,2,2,2,2,1,1
6861,0,37.000000,101.90,1,0,0,1,2,1,0,2,2,0,2,2,0,1,0
3266,0,29.000000,70.75,1,1,1,1,0,1,0,0,0,0,0,0,0,1,2


In [61]:
df_test.head()

,SeniorCitizen,tenure,MonthlyCharges,Partner_num,Dependents_num,PhoneService_num,MultipleLines_num,InternetService_num,OnlineSecurity_num,OnlineBackup_num,DeviceProtection_num,TechSupport_num,StreamingTV_num,StreamingMovies_num,Contract_num,PaperlessBilling_num,PaymentMethod_num
0,0,23.0,20.00,1,1,1,0,2,1,1,1,1,1,1,1,1,1
1,1,61.0,99.00,1,1,1,2,1,0,2,0,0,2,2,1,0,4
2,0,36.0,84.75,0,0,1,2,1,0,0,0,0,0,2,0,1,2
3,0,61.0,61.45,1,1,0,1,0,0,2,2,2,2,2,2,1,0
4,0,47.0,20.55,0,0,1,0,2,1,1,1,1,1,1,2,1,3


## Treinando o Modelo AdaBoost

In [62]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.datasets import make_classification

In [83]:
dt = AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None, learning_rate=1.0, n_estimators=200)

X_train = df_train.drop('Churn', axis=1)
y_train = df_train['Churn']

dt.fit(X_train, y_train)

AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None, learning_rate=1.0,
                   n_estimators=200, random_state=None)

In [84]:
my_predictions = dt.predict(df_test)
submission['Churn'] = my_predictions
submission.to_csv('/gdrive/My Drive/Python/ChurnAdaBoost2b.csv', index=False)

## Treinando o Modelo GradientBoosting

In [66]:
from sklearn.datasets import make_classification
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split

In [72]:
X_train = df_train.drop('Churn', axis=1)
y_train = df_train['Churn']

X_test = df_test
y_test = df_test['SeniorCitizen']

In [67]:
X_train.shape

(5634, 17)

In [68]:
X_test.shape

(1409, 19)

In [69]:
y_train.shape

(5634,)

In [70]:
y_test.shape

(1409,)

In [73]:
dt = GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.409, loss='deviance', max_depth=1,
                           max_features=1.0, max_leaf_nodes=None,
                           min_impurity_decrease=0.0005,
                           min_impurity_split=None, min_samples_leaf=1,
                           min_samples_split=4, min_weight_fraction_leaf=0.0,
                           n_estimators=60, n_iter_no_change=None,
                           presort='deprecated', random_state=123,
                           subsample=0.4, tol=0.0001, validation_fraction=0.1,
                           verbose=0, warm_start=False)

dt.fit(X_train, y_train)

my_predictions = dt.predict(X_test)
submission['Churn'] = my_predictions
submission.to_csv('/gdrive/My Drive/Python/ChurnGradientBoost2.csv', index=False)

## Treinando o Modelo LogisticRegression

In [75]:
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression

In [76]:
X_train = df_train.drop('Churn', axis=1)
y_train = df_train['Churn']

X_test = df_test
y_test = df_test['SeniorCitizen']

In [77]:
dt = LogisticRegression(C=7.007000000000001, class_weight={}, dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=1000, multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=123, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

dt.fit(X_train, y_train)

my_predictions = dt.predict(X_test)
submission['Churn'] = my_predictions
submission.to_csv('/gdrive/My Drive/Python/ChurnLogisticRegression2.csv', index=False)

## Treinando o Modelo CatBoostingClassifier

In [ ]:
!pip install catboost

In [74]:
from catboost import CatBoostClassifier

In [ ]:
dt = CatBoostClassifier(auto_class_weights= None,
 #bayesian_matrix_reg= 0.10000000149011612,
 best_model_min_trees= 1,
 boost_from_average= False,
 boosting_type= 'Plain',
 bootstrap_type= 'MVS',
 border_count= 254,
 class_names= [0, 1],
 classes_count= 0,
 depth= 4,
 eval_metric= 'Logloss',
 feature_border_type= 'GreedyLogSum',
 grow_policy= 'SymmetricTree',
 iterations= 290,
 l2_leaf_reg= 30,
 leaf_estimation_backtracking= 'AnyImprovement',
 leaf_estimation_iterations= 10,
 leaf_estimation_method= 'Newton',
 learning_rate= 0.029999999329447743,
 loss_function= 'Logloss',
 max_leaves= 16,
 min_data_in_leaf= 1,
 model_shrink_mode= 'Constant',
 model_shrink_rate= 0,
 model_size_reg= 0.5,
 nan_mode= 'Min',
 penalties_coefficient= 1,
 posterior_sampling= False,
 random_seed= 123,
 random_strength= 0.5,
 rsm= 1,
 sampling_frequency= 'PerTree',
 score_function= 'Cosine',
 sparse_features_conflict_fraction= 0,
 subsample= 0.800000011920929,
 task_type= 'CPU',
 use_best_model= False)

dt.fit(X_train, y_train)
my_predictions = dt.predict(X_test)
submission['Churn'] = my_predictions
submission.to_csv('/gdrive/My Drive/Python/ChurnCatBoosting.csv', index=False)

In [82]:
dt = CatBoostClassifier(auto_class_weights= 'None',
 #bayesian_matrix_reg= 0.10000000149011612,
 best_model_min_trees= 1,
 boost_from_average= False,
 boosting_type= 'Plain',
 bootstrap_type= 'MVS',
 border_count= 254,
 class_names= [0, 1],
 classes_count= 0,
 depth= 2,
 eval_metric= 'Logloss',
 feature_border_type= 'GreedyLogSum',
 grow_policy= 'SymmetricTree',
 iterations= 300,
 l2_leaf_reg= 6,
 leaf_estimation_backtracking= 'AnyImprovement',
 leaf_estimation_iterations= 10,
 leaf_estimation_method= 'Newton',
 learning_rate= 0.029999999329447743,
 loss_function= 'Logloss',
 max_leaves= 4,
 min_data_in_leaf= 1,
 model_shrink_mode= 'Constant',
 model_shrink_rate= 0,
 model_size_reg= 0.5,
 nan_mode= 'Min',
 penalties_coefficient= 1,
 posterior_sampling= False,
 random_seed= 123,
 random_strength= 0.30000001192092896,
 rsm= 1,
 sampling_frequency= 'PerTree',
 score_function= 'Cosine',
 sparse_features_conflict_fraction= 0,
 subsample= 0.800000011920929,
 task_type= 'CPU',
 use_best_model= False)

dt.fit(X_train, y_train)
my_predictions = dt.predict(X_test)
submission['Churn'] = my_predictions
submission.to_csv('/gdrive/My Drive/Python/ChurnCatBoosting2b.csv', index=False)

0:	learn: 0.6749898	total: 2.44ms	remaining: 731ms
1:	learn: 0.6590336	total: 5.25ms	remaining: 782ms
2:	learn: 0.6434683	total: 7.5ms	remaining: 742ms
3:	learn: 0.6286945	total: 9.67ms	remaining: 716ms
4:	learn: 0.6154486	total: 11.8ms	remaining: 696ms
5:	learn: 0.6028920	total: 13.9ms	remaining: 682ms
6:	learn: 0.5913561	total: 16.3ms	remaining: 684ms
7:	learn: 0.5809587	total: 18.6ms	remaining: 678ms
8:	learn: 0.5713858	total: 20.7ms	remaining: 669ms
9:	learn: 0.5622698	total: 23.4ms	remaining: 678ms
10:	learn: 0.5551386	total: 25.5ms	remaining: 670ms
11:	learn: 0.5482185	total: 27.7ms	remaining: 664ms
12:	learn: 0.5419706	total: 29.8ms	remaining: 657ms
13:	learn: 0.5357556	total: 31.9ms	remaining: 652ms
14:	learn: 0.5290330	total: 34.2ms	remaining: 649ms
15:	learn: 0.5235088	total: 36.4ms	remaining: 647ms
16:	learn: 0.5183687	total: 38.6ms	remaining: 643ms
17:	learn: 0.5139005	total: 40.9ms	remaining: 641ms
18:	learn: 0.5094755	total: 43.1ms	remaining: 637ms
19:	learn: 0.5052890	to